In [1]:
!pip install pytesseract


In [2]:
# 🔗 STEP 1: Mount Google Drive
from google.colab import drive
import os
import cv2
import numpy as np
import pytesseract
import csv

drive.mount('/content/drive')

# 📁 STEP 2: Set your image folder path (update if needed!)
image_folder = "/content/drive/My Drive/4th Year/DS 4002/Project 3/Doppler_Images"
output_csv = "/content/drive/My Drive/4th Year/DS 4002/Project 3/ocr_results.csv"


Mounted at /content/drive


In [3]:
# 🧠 STEP 3: Check if folder exists
if not os.path.exists(image_folder):
    print("🚫 Folder not found. Double-check the path!")
else:
    print("✅ Folder found! Starting OCR...\n")

✅ Folder found! Starting OCR...



In [4]:

# 🧬 STEP 4: Run OCR on each image + save results
ocr_config = r'--psm 6 -c tessedit_char_whitelist=0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ/+-μ%.'

with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Filename', 'DetectedText', 'X', 'Y'])

    for filename in sorted(os.listdir(image_folder)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(image_folder, filename)
            print(f"\n📄 Processing: {filename}")

            image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"❌ Could not read: {filename}")
                continue

            h, w = image.shape
            crop_width = int(w * 0.1)
            scale_bar = image[:, w - crop_width:]

            eq = cv2.equalizeHist(scale_bar)
            _, thresh = cv2.threshold(eq, 160, 255, cv2.THRESH_BINARY)
            inverted = cv2.bitwise_not(thresh)

            data = pytesseract.image_to_data(inverted, config=ocr_config, output_type=pytesseract.Output.DICT)

            results = []
            for i in range(len(data['text'])):
                text = data['text'][i].strip()
                if text:
                    x, y, bw, bh = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
                    cx, cy = x + bw // 2, y + bh // 2
                    full_x = cx + (w - crop_width)
                    results.append((text, full_x, cy))
                    writer.writerow([filename, text, full_x, cy])
                    print(f"  → '{text}' at (x={full_x}, y={cy})")

print("\n✅ Done! Results saved to:", output_csv)


📄 Processing: 1.jpeg
  → 'i' at (x=1343, y=11)
  → 'a' at (x=1410, y=466)
  → 'ec-2022920AM' at (x=1412, y=556)

📄 Processing: 10.png
  → '100' at (x=1007, y=116)
  → 'cmis' at (x=1013, y=196)
  → '.' at (x=978, y=253)
  → '-100' at (x=1010, y=271)
  → '--200' at (x=998, y=335)
  → '300' at (x=1010, y=409)
  → 'stoom' at (x=993, y=490)

📄 Processing: 11.png
  → 'mis' at (x=1001, y=109)
  → '-1.0' at (x=1001, y=178)
  → '-2.0' at (x=1001, y=248)
  → '-3.0' at (x=1001, y=317)
  → '4.0' at (x=1001, y=385)
  → 'Spin' at (x=1000, y=466)

📄 Processing: 12.png
  → '100' at (x=999, y=112)
  → 'ems' at (x=1002, y=189)
  → '--100' at (x=1002, y=249)
  → '200' at (x=1011, y=302)
  → '300' at (x=1011, y=365)
  → '400' at (x=1007, y=426)
  → 'sobom' at (x=991, y=489)

📄 Processing: 13.png
  → '-100' at (x=1018, y=121)
  → '-cm/s' at (x=1025, y=198)
  → '--100' at (x=1021, y=275)
  → '--200' at (x=1021, y=351)
  → '--300' at (x=1021, y=429)
  → '4bpm' at (x=1018, y=487)

📄 Processing: 14.jpeg
  → '